In [1]:
import requests
import pandas as pd
import numpy as np
import json

from util import convert_amount, parse_currency

In [15]:
r = requests.get('http://localhost:8000/products?format=json')
shop_df = pd.DataFrame(json.loads(r.content.decode()))
shop_df['name'] = [i.lower() for i in shop_df['name']]
shop_df.head()
shop_df_ = shop_df.copy()
cols = ['name', 'price', 'currency', 'amount']
shop_df = pd.DataFrame(columns=[*cols, 'unit'])
for idx, name, price, currency, amount in enumerate(shop_df_[cols].to_numpy()):
    amount, unit = convert_amount(amount)
    price = parse_currency(price, currency)
    shop_df.loc[idx] = [name, price, amount, unit]

ValueError: not enough values to unpack (expected 5, got 2)

In [ ]:
shop_df

In [8]:
filepath = 'shopping_lists/shoppingList13.json'
with open(filepath) as f:
    shopping_list = json.load(f)

In [9]:
def get_items(shopping_list):
    items_df = pd.DataFrame([i[0] for i in pd.DataFrame(shopping_list['aisles'])['items']])
    qties = pd.DataFrame(list(items_df['measures']))['metric']
    items_df.drop(['measures', 'usages', 'pantryItem', 'cost'], axis=1, inplace=True)
    items_df['amount'] = [i['amount'] for i in qties]
    items_df['unit'] = [i['unit'] for i in qties]
    items_df['name'] = items_df['name'].map(str.split)
    to_drop = ['quick', 'canned', 'cooking', 'fat', 'ground', 'leaves', 'pasta',
               'plain', 'cooked', 'full-fat', 'pieces', 'scrambled', 'frozen', 'creamy']
    items_df['name'] = [[j for j in i if j not in to_drop] for i in items_df['name']]
    return items_df

In [11]:
items_df = get_items(shopping_list)

In [12]:
items_df

,id,name,aisle,ingredientId,amount,unit
0,600627,[sprite],Beverages,14144,1.0,
1,600635,"[burrito, wrap]",Ethnic Foods,10118364,1.0,
2,600637,[tuna],Seafood,10015121,4.0,
3,600651,"[mat, beans]",Pasta and Rice,99144,1.0,
4,600655,[tomato],Produce,11529,1.0,
5,600659,[water],Pantry Items,14412,2.0,
6,600661,[chicken],Meat,5006,1.0,
7,600665,[fries],Frozen,11408,2.0,
8,600669,[bread],Bakery/Bread,18064,1.0,
9,600671,"[potato, chips]",Savory Snacks,19411,5.0,


In [13]:
matches_df = pd.DataFrame()
for product in items_df.iloc:
    regex = ''.join([f'(?=.*{i})' for i in product['name']])   
    entries = shop_df['name'].str.contains(regex)    
    if entries.any():
        matches = shop_df[entries]
        print(f"appending {' '.join(product['name'])}")
        matches_df = matches_df.append(matches)
    else:
        print(f"no such product: {' '.join(product['name'])}")

no such product: sprite
no such product: burrito wrap
appending tuna
appending mat beans
appending tomato
appending water
appending chicken
appending fries
appending bread
appending potato chips
appending tea
appending granola
no such product: m&m candy
appending yogurt
no such product: pizza mix


In [85]:
shop_df[shop_df['name'].str.contains('(?=.*)(?=.*pasta)')]

,store,name,img_url,price,currency,amount,unit
736,sainsburys,dolmio bolognese pasta sauce original 500g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.4,GBP,500g,unit
912,sainsburys,dolmio bolognese pasta sauce original 750g,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.0,GBP,750g,unit


### add to the database:
    - ground cumin / cumin
    - hot sauce
    - almonds, agave
    - salsa
    - tahini
    - almond milk
    - worcestershire sauce
    - flax egg
    - maple syrup
    - lemon juice
    - garlic powder
    - gluten-free
    - lime juice
    - pesto
    - mozzarella
    - cornstarch
    - green bell pepper
    - red pepper flakes
    - pumpking seeds
    - dressings (caeasar etc)
    - cashews
    - dijon mustard
    - chili sauce
    - oats
    - peanut butter
    - tortilas
    - vanilla extract
    - starch
    - vinegar (white, apple cider)
    - soda, candy

### change

- convert 4x5 to 20 etc
- convert price to one common unit (pence)
- if ground and meat then mince

In [39]:
shop_df[shop_df['amount'] == 'Loose']

,store,name,img_url,price,currency,amount,unit
0,sainsburys,sainsbury's fairtrade bananas loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,73.00,p,Loose,kg
1,sainsburys,sainsbury's carrots loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,41.00,p,Loose,kg
3,sainsburys,sainsbury's onions loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Sa...,85.00,p,Loose,kg
4,sainsburys,sainsbury's baking potatoes loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,64.00,p,Loose,kg
11,sainsburys,sainsbury's broccoli loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.60,GBP,Loose,kg
12,sainsburys,sainsbury's red onions loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Sa...,1.00,GBP,Loose,kg
13,sainsburys,sainsbury's closed cup white mushrooms loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,2.75,GBP,Loose,kg
17,sainsburys,sainsbury's courgettes loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Sa...,2.00,GBP,Loose,kg
18,sainsburys,sainsbury's sweet potatoes loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.10,GBP,Loose,kg
25,sainsburys,sainsbury's parsnips loose,https://www.sainsburys.co.uk/wcsstore7.59.1/Ex...,1.15,GBP,Loose,kg
